In [1]:
from google.colab import files
import pandas as pd

path = "/content/drive/MyDrive/ind-market-dataset/ind_niftytotalmarket_list.csv"
#path = "/content/drive/MyDrive/ind-market-dataset/ind_niftytotalmarket_list_test.xlsx"

df=pd.read_csv(path)
#df = pd.read_excel(path)
df.head(5)

,Company Name,Industry,Symbol,Series,ISIN Code
0,360 ONE WAM Ltd.,Financial Services,360ONE,EQ,INE466L01038
1,3M India Ltd.,Diversified,3MINDIA,EQ,INE470A01017
2,ABB India Ltd.,Capital Goods,ABB,EQ,INE117A01022
3,ACC Ltd.,Construction Materials,ACC,EQ,INE012A01025
4,ACME Solar Holdings Ltd.,Power,ACMESOLAR,EQ,INE622W01025


In [2]:
# Install necessary packages
!pip install yfinance pandas tqdm datetime --quiet

import pandas as pd
import yfinance as yf
#from datetime import datetime
from datetime import date
from tqdm.notebook import tqdm

# STEP 1: Read your Nifty stock symbols from CSV
# Make sure your CSV has a column named 'Symbol'
nifty_df = df
nifty_symbols = nifty_df['Symbol'].tolist()

# STEP 2: Set up year selection
today = date.today()
current_year = today.year
last_year = current_year - 1
start_date = f"{last_year}-01-01"
end_date = f"{last_year}-12-31"

print(today, current_year, last_year, start_date, end_date)

def fibonacci_pivots(h, l, c):
    pp = (h + l + c) / 3
    return {
        'PP': float(pp),
        'R1': float(pp + (h - l) * 0.382),
        'R2': float(pp + (h - l) * 0.618),
        'R3': float(pp + (h - l)),
        'S1': float(pp - (h - l) * 0.382),
        'S2': float(pp - (h - l) * 0.618),
        'S3': float(pp - (h - l))
    }

below_S2 = []
below_S3 = []
below_PP = []
below_R3 = []

# STEP 3: For each stock, download historical data and apply Fibonacci filter
for symbol in tqdm(nifty_symbols):
    ticker = symbol + ".NS"
    try:
        # Download ALL data since last year's Jan 1
        df = yf.download(ticker, start=start_date, end=today, interval = '1d', progress=False);
        if df.empty or (len(df[df.index.year == last_year]) == 0) or (len(df[df.index.year == current_year]) == 0):
            continue

        # Get last year's yearly OHLC
        df_last_year = df[df.index.year == last_year]
        print(df_last_year)
        o = df_last_year.iloc[0]['Open']         # Opening price first trading day
        h = df_last_year['High'].max()           # High of last year
        l = df_last_year['Low'].min()            # Low of last year
        c = df_last_year.iloc[-1]['Close']       # Close of last trading day of Dec
        print(o,h,l,c)

        pivots = fibonacci_pivots(h, l, c)

        # Get most recent close for this year
        df_this_year = df[df.index.year == current_year]
        current_close = float(df_this_year.iloc[-1]['Close'])
        print (f"PP:{pivots['PP']}, S2:{pivots['S2']}, S3:{pivots['S3']}")

        if current_close < pivots['S2']:
            below_S2.append(symbol)
        if current_close < pivots['S3']:
            below_S3.append(symbol)
        if current_close < pivots['PP']:
            below_PP.append(symbol)
        if current_close < pivots['R3']:
            below_R3.append(symbol)

    except Exception as e:
        print(f"Error for {symbol}: {str(e)}")
        continue


Output hidden; open in https://colab.research.google.com to view.

In [3]:
print("Stocks currently below S2:", below_S2)
print("Stocks currently below S3:", below_S3)
print("Stocks currently below PP:", below_PP)

Stocks currently below S2: ['ABFRL', 'ALLCARGO', 'BSOFT', 'FIRSTCRY', 'CEIGALL', 'CELLO', 'CENTRALBK', 'CLEAN', 'CONCOR', 'CYIENTDLM', 'CYIENT', 'DEEPAKNTR', 'EASEMYTRIP', 'EMBDL', 'FCL', 'FIVESTAR', 'GATEWAY', 'HAPPSTMNDS', 'IIFLCAPS', 'INDIGOPNTS', 'INFIBEAM', 'IGIL', 'IKS', 'JAIBALAJI', 'JAICORPLTD', 'JINDWORLD', 'KNRCON', 'MAHLIFE', 'MANINFRA', 'NOCIL', 'NTPCGREEN', 'ORCHPHARMA', 'PATELENG', 'PRAJIND', 'RAYMONDLSL', 'RAYMOND', 'RESPONIND', 'ROUTE', 'SHAREINDIA', 'RENUKA', 'SIEMENS', 'SONACOMS', 'SONATSOFTW', 'SWSOLAR', 'SULA', 'TVSSCS', 'TCS', 'TATATECH', 'TEAMLEASE', 'UCOBANK', 'UNIMECH', 'VBL', 'MANYAVAR']
Stocks currently below S3: ['ABFRL', 'FIRSTCRY', 'CYIENT', 'IGIL', 'IKS', 'JINDWORLD', 'ROUTE', 'SHAREINDIA', 'SONACOMS', 'TEAMLEASE', 'UNIMECH']
Stocks currently below PP: ['3MINDIA', 'ABB', 'ACC', 'AGI', 'AIAENG', 'AWL', 'AARTIIND', 'AAVAS', 'ACE', 'ADANIENSOL', 'ADANIENT', 'ADANIGREEN', 'ATGL', 'ABFRL', 'ABREL', 'ADVENZYMES', 'AEGISLOG', 'AETHER', 'AFCONS', 'AHLUCONT', 'AJAN